Importing the dependencies 

In [1]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier 
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier, VotingClassifier, StackingClassifier, HistGradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier, RidgeClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

Data collection and Processing 

In [2]:
#loading the water quality dataset to pandas DataFrame
water_data= pd.read_csv('/kaggle/input/water-prediction/water quality/water_potability.csv')

In [3]:
#print first 5 rows 

water_data.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [4]:
#print last 5 rows 

water_data.tail()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
3271,4.668102,193.681735,47580.991603,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3272,7.808856,193.553212,17329.802160,8.061362,NaN,392.449580,19.903225,NaN,2.798243,1
3273,9.419510,175.762646,33155.578218,7.350233,NaN,432.044783,11.039070,69.845400,3.298875,1
3274,5.126763,230.603758,11983.869376,6.303357,NaN,402.883113,11.168946,77.488213,4.708658,1
3275,7.874671,195.102299,17404.177061,7.509306,NaN,327.459760,16.140368,78.698446,2.309149,1


In [5]:
#number of rows and columns in the dataset
water_data.shape

(3276, 10)

In [6]:
# information about the dataset
water_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3276 entries, 0 to 3275
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ph               2785 non-null   float64
 1   Hardness         3276 non-null   float64
 2   Solids           3276 non-null   float64
 3   Chloramines      3276 non-null   float64
 4   Sulfate          2495 non-null   float64
 5   Conductivity     3276 non-null   float64
 6   Organic_carbon   3276 non-null   float64
 7   Trihalomethanes  3114 non-null   float64
 8   Turbidity        3276 non-null   float64
 9   Potability       3276 non-null   int64  
dtypes: float64(9), int64(1)
memory usage: 256.1 KB


In [7]:
#check if there is any missing valuse
water_data.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [8]:
# This code will fill the missing values in the 'ph', 'Sulfate', and 'Trihalomethanes' columns with the median of their respective columns

# Fill missing values with the median of each column
water_data['ph'] = water_data['ph'].fillna(water_data['ph'].median())
water_data['Sulfate'] = water_data['Sulfate'].fillna(water_data['Sulfate'].median())
water_data['Trihalomethanes'] = water_data['Trihalomethanes'].fillna(water_data['Trihalomethanes'].median())

# Verify the changes
water_data.isnull().sum()



ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [9]:
# Some statical mesures about this dataset

water_data.describe().T



,count,mean,std,min,25%,50%,75%,max
ph,3276.0,7.074194,1.470040,0.000000,6.277673,7.036752,7.870050,14.000000
Hardness,3276.0,196.369496,32.879761,47.432000,176.850538,196.967627,216.667456,323.124000
Solids,3276.0,22014.092526,8768.570828,320.942611,15666.690297,20927.833607,27332.762127,61227.196008
Chloramines,3276.0,7.122277,1.583085,0.352000,6.127421,7.130299,8.114887,13.127000
Sulfate,3276.0,333.608364,36.143851,129.000000,317.094638,333.073546,350.385756,481.030642
Conductivity,3276.0,426.205111,80.824064,181.483754,365.734414,421.884968,481.792304,753.342620
Organic_carbon,3276.0,14.284970,3.308162,2.200000,12.065801,14.218338,16.557652,28.300000
Trihalomethanes,3276.0,66.407478,15.769958,0.738000,56.647656,66.622485,76.666609,124.000000
Turbidity,3276.0,3.966786,0.780382,1.450000,3.439711,3.955028,4.500320,6.739000
Potability,3276.0,0.390110,0.487849,0.000000,0.000000,0.000000,1.000000,1.000000


In [10]:
# checking the distribution of Target Variable

water_data['Potability'].value_counts()

Potability
0    1998
1    1278
Name: count, dtype: int64

**0 --> Not Safe for Humans to Drink**

**1 --> Safe for Humans to Drink**

Splitting the Features and Targets 

In [11]:
X = water_data.drop(columns= 'Potability', axis=1)
Y = water_data['Potability']

# checking the Changes in X and 

print(X)

            ph    Hardness        Solids  Chloramines     Sulfate  \
0     7.036752  204.890455  20791.318981     7.300212  368.516441   
1     3.716080  129.422921  18630.057858     6.635246  333.073546   
2     8.099124  224.236259  19909.541732     9.275884  333.073546   
3     8.316766  214.373394  22018.417441     8.059332  356.886136   
4     9.092223  181.101509  17978.986339     6.546600  310.135738   
...        ...         ...           ...          ...         ...   
3271  4.668102  193.681735  47580.991603     7.166639  359.948574   
3272  7.808856  193.553212  17329.802160     8.061362  333.073546   
3273  9.419510  175.762646  33155.578218     7.350233  333.073546   
3274  5.126763  230.603758  11983.869376     6.303357  333.073546   
3275  7.874671  195.102299  17404.177061     7.509306  333.073546   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  
0       564.308654       10.379783        86.990970   2.963135  
1       592.885359       15.180013       

In [12]:
print(Y)

0       0
1       0
2       0
3       0
4       0
       ..
3271    1
3272    1
3273    1
3274    1
3275    1
Name: Potability, Length: 3276, dtype: int64


In [13]:
# Data Preprocessing
def preprocess_data(data):
    # Extract features and target variable
    features = data.drop(columns='Potability')
    target = data['Potability']
    
    # Perform log transformation on features
    log_transformed_features = np.log1p(features)
    
    # Balance the dataset using SMOTE
    smote = SMOTE(random_state=42)
    balanced_features, balanced_target = smote.fit_resample(log_transformed_features, target)
    
    # Scale the features using RobustScaler
    scaler = RobustScaler()
    scaled_features = scaler.fit_transform(balanced_features)
    
    return train_test_split(scaled_features, balanced_target, test_size=0.2, random_state=42)


In [14]:
# Define and Train Models
def train_models(X_train, y_train, X_test, y_test):
    classifiers = {
        'Logistic Regression': LogisticRegression(),
        'Decision Tree': DecisionTreeClassifier(),
        'Random Forest': RandomForestClassifier(),
        'Support Vector Machine': SVC(),
        'K-Nearest Neighbors': KNeighborsClassifier(),
        'Bagging': BaggingClassifier(),
        'AdaBoost': AdaBoostClassifier(),
        'Voting': VotingClassifier(estimators=[
            ('lr', LogisticRegression()), 
            ('rf', RandomForestClassifier()), 
            ('svc', SVC())
        ]),
        'Stacking': StackingClassifier(estimators=[
            ('lr', LogisticRegression()), 
            ('rf', RandomForestClassifier()), 
            ('svc', SVC())
        ], final_estimator=LogisticRegression()),
        'Linear SVM': LinearSVC(),
        'SGDClassifier': SGDClassifier(),
        'RidgeClassifier': RidgeClassifier(),
        'HistGradientBoosting': HistGradientBoostingClassifier(),
        'LightGBM': LGBMClassifier()
    }

    best_accuracy = 0
    best_model_name = ''
    best_model = None

    for model_name, model in classifiers.items():
        model.fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model_name = model_name
            best_model = model

    print(f"Best Model: {best_model_name}")
    print(f"Accuracy: {best_accuracy:.3f}")

In [15]:
# Hyperparameter Tuning for Random Forest
def tune_random_forest(X_train, y_train, X_test, y_test):
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }

    rf = RandomForestClassifier()
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    best_params = grid_search.best_params_
    best_rf = grid_search.best_estimator_

    y_pred = best_rf.predict(X_test)
    test_data_accuracy = accuracy_score(y_test, y_pred)

    print(f"Best Parameters: {best_params}")
    print(f"Test Data Accuracy with Tuned Random Forest: {test_data_accuracy:.3f}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

# Main Execution
X_train, X_test, y_train, y_test = preprocess_data(water_data)
print("Training set shape:", X_train.shape, y_train.shape)
print("Test set shape:", X_test.shape, y_test.shape)

train_models(X_train, y_train, X_test, y_test)
tune_random_forest(X_train, y_train, X_test, y_test)

Training set shape: (3196, 9) (3196,)
Test set shape: (800, 9) (800,)


/opt/conda/lib/python3.10/site-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


[LightGBM] [Info] Number of positive: 1592, number of negative: 1604
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000408 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2295
[LightGBM] [Info] Number of data points in the train set: 3196, number of used features: 9
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.498123 -> initscore=-0.007509
[LightGBM] [Info] Start training from score -0.007509
Best Model: Random Forest
Accuracy: 0.740
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.9s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   0.8s
[CV] END bootstrap=True, max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   1.5s
[CV] END bootstrap=True, max_depth=None, min_samp